In [36]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dropout, RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomCrop, MaxPooling2D, Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.layers import BatchNormalization
from numpy import mean, std 
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

# Loading Hard Test Data

data_train = np.load('data.npy')
labels_train = np.load('labels.npy')

print(data_train.shape, labels_train.shape)

(90000, 400) (400,)


In [37]:
import cv2
hard_test_data_length = data_train.shape[1]
def load_hard_test_data(length):
    # load dataset
    data_rgb = []
    kernel = np.ones((4,4),np.uint8)
    for i in range(hard_test_data_length):
        data_rgb.append(data_train[:,i].reshape(300,300))
        data_rgb[i] = cv2.medianBlur(data_rgb[i], 3)
        data_rgb[i] = cv2.morphologyEx(data_rgb[i], cv2.MORPH_OPEN, kernel)
        data_rgb[i] = cv2.resize(data_rgb[i], (50,50), interpolation=cv2.INTER_AREA)
    data_rgb = np.array(data_rgb)
    print(data_rgb.shape)
    
    # reshape dataset to have a single channel
    data_rgb = data_rgb.reshape((data_rgb.shape[0], 50,50, 1))
    
    # one hot encode target values
    labels = to_categorical(labels_train)
    print(data_rgb.shape, labels.shape)
    return data_rgb, labels

In [38]:
# scale pixels
def prep_pixels(data):
    # convert from integers to floats
    data = data.astype('float32')
    
    # he_uniformize to range 0-1
    data_norm = (data) / 255.0
    
    # return he_uniformized images
    return data_norm

In [39]:
model = load_model('Final_CNN')

In [40]:
hard_test_data, hard_test_label = load_hard_test_data(hard_test_data_length)
    
# prepare pixel data
hard_test_data = prep_pixels(hard_test_data)

(400, 50, 50)
(400, 50, 50, 1) (400, 10)


In [41]:
pred = model.predict(hard_test_data)
print(hard_test_label.shape, pred.shape)
prediction = []
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(hard_test_label, axis=1)
for i in range(hard_test_data_length):
    if max(pred[i]) < 0.9:
        y_pred[i] = -1
    prediction.extend([y_true[i], y_pred[i]])
prediction = np.array(prediction)
prediction = np.reshape(prediction, (hard_test_data_length,2))
df = pd.DataFrame (prediction, columns = ['TRUE LABEL', 'LABEL PREDICTED'])
df

(400, 10) (400, 10)


,TRUE LABEL,LABEL PREDICTED
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
395,5,5
396,6,6
397,7,7
398,8,8


In [42]:
print("CONFUSION MATRIX: ")
print(confusion_matrix(y_true,y_pred))

CONFUSION MATRIX: 
[[ 0  0  0  0  0  0  0  0  0  0  0]
 [ 1 39  0  0  0  0  0  0  0  0  0]
 [ 0  0 40  0  0  0  0  0  0  0  0]
 [ 2  0  0 38  0  0  0  0  0  0  0]
 [ 0  0  0  0 40  0  0  0  0  0  0]
 [ 0  0  0  0  0 40  0  0  0  0  0]
 [ 0  0  0  0  0  0 40  0  0  0  0]
 [ 0  0  0  0  0  0  0 40  0  0  0]
 [ 0  0  0  0  0  0  0  0 40  0  0]
 [ 0  0  0  0  0  0  0  0  0 40  0]
 [ 0  0  0  0  0  0  0  0  0  0 40]]


In [43]:
print("ACCUARCY : - ",accuracy_score(y_true, y_pred))

ACCUARCY : -  0.9925
